## The purpose of this notebook is to turn the datasets returned from the lag creator to be grouped by the day or hour and add the tweet text for that day or hour

In [1]:
#imports
import pandas as pd
import os
import gc
import sys

In [2]:
#get data with following lag
granularity = "days"
lag = 7
# analyser = "TextBlob"
analyser = "vader"

#how to group - by "day" or "hour"
group_by = "day"

#if to use cleaned data
cleaned = True
#if with polarity
with_polarity = True

In [3]:
#prepare filename to read
folder = "./../datasets/tweets_and_prices/vader" if with_polarity else  "./../datasets/tweets_and_prices/cleaned";

#if cleaned and with polarity add the cleaned dir
if(cleaned and with_polarity):
    folder = folder+"/cleaned"

print(folder)    
    
filename = folder+"/final_data_lag_"+granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"

./../datasets/tweets_and_prices/vader/cleaned


## Getting tweet volumes

In [4]:
#load all tweets if cleaned

#prepare original filename to read
original_folder = "./../datasets/tweets_and_prices/vader"
original_filename = original_folder+"/final_data_lag_"+granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"

tweets_original = pd.read_csv(original_filename) if cleaned else pd.read_csv(filename)

if(cleaned):
    tweets_original["dt"] = pd.to_datetime(tweets_original['timestamp'])
    tweets_original = tweets_original.drop(['timestamp'], axis=1)
    
#floor datetimes to day or hour
tweets_original['DateTime'] = tweets_original['dt'].dt.floor(group_by[0])

C:\Users\dell\Anaconda3\envs\fyp\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
original_folder

'./../datasets/tweets_and_prices/vader'

In [6]:
#group tweet volumes daily
grouped_volumes = tweets_original.groupby(pd.Grouper(key='dt',freq=group_by[0])).size().reset_index(name='tweet_vol')

In [7]:
#set index to datetime
grouped_volumes.index = pd.to_datetime(grouped_volumes.index)
grouped_volumes = grouped_volumes.set_index('dt')

In [8]:
#set type of volumes to float
grouped_volumes['tweet_vol'] = grouped_volumes['tweet_vol'].astype(float)

In [9]:
grouped_volumes.head()

,tweet_vol
dt,
2018-08-24 00:00:00+00:00,506.0
2018-08-25 00:00:00+00:00,857.0
2018-08-26 00:00:00+00:00,796.0
2018-08-27 00:00:00+00:00,885.0
2018-08-28 00:00:00+00:00,1012.0


In [10]:
tweets_original.head()

,Unnamed: 0,id,fullname,url,replies,likes,retweets,text,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,neg_pol,neu_pol,pos_pol,compound,dt,DateTime
0,0,1.132977e+18,Abdiel kamdem,NaN,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,8459.94,5.847993,49460.152431,8457.629064,0.00,1.000,0.000,0.0000,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
1,1,1.132977e+18,Bitcointe,NaN,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,8459.94,5.847993,49460.152431,8457.629064,0.04,0.876,0.085,0.3818,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
2,2,1.132977e+18,Bran - 3 Eyed Raven,NaN,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,8459.94,5.847993,49460.152431,8457.629064,0.00,1.000,0.000,0.0000,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
3,3,1.132977e+18,J. Scardina,NaN,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,8459.94,5.847993,49460.152431,8457.629064,0.00,1.000,0.000,0.0000,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00
4,4,1.132977e+18,Muhammad Mursaleen,NaN,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,8459.94,5.847993,49460.152431,8457.629064,0.00,0.857,0.143,0.3612,2019-05-27 11:49:00+00:00,2019-05-27 00:00:00+00:00


In [11]:
close_prices = pd.Series(tweets_original.loc[tweets_original.groupby('DateTime').dt.idxmax()].set_index('DateTime')['Close'])

#get final prices of day
# tweets_df["Close"] = pd.Series(tweets_original.loc[tweets_original.groupby('DateTime').dt.idxmax()]["Close"]).values
# df_copy["Open"] = pd.Series(df.loc[df.groupby('DateTime').dt.idxmin()]["Open"]).values
# df_copy["High"] = pd.Series(df.loc[df.groupby('DateTime').Close.idxmax()]["High"]).values
# df_copy["Low"] = pd.Series(df.loc[df.groupby('DateTime').Close.idxmin()]["Low"]).values

In [12]:
del tweets_original

In [13]:
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

               grouped_volumes:  7.2 KiB
                  close_prices:  7.2 KiB
                             _:  3.0 KiB
                           _10:  3.0 KiB
                           _i4:  659.0 B
                           _ii:  590.0 B
                          _i11:  590.0 B
                          _i13:  576.0 B
                           _i3:  445.0 B
                           _i2:  281.0 B


## Group tweets

In [14]:
#read data
df = pd.read_csv(filename)

C:\Users\dell\Anaconda3\envs\fyp\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (2,3,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
df.head()

,Unnamed: 0,id,fullname,url,timestamp,replies,likes,retweets,text,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,neg_pol,neu_pol,pos_pol,compound
0,0,NaN,NaN,NaN,2018-08-24 11:04:00+00:00,NaN,NaN,NaN,NaN,6936.26,1.001363,6944.466619,6935.011702,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,2018-08-24 11:05:00+00:00,NaN,NaN,NaN,NaN,6944.90,10.379585,72022.107341,6938.823172,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,2018-08-24 11:06:00+00:00,NaN,NaN,NaN,NaN,6949.89,1.264825,8789.841563,6949.454444,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,2018-08-24 11:07:00+00:00,NaN,NaN,NaN,NaN,6950.61,1.948446,13550.262119,6954.393879,NaN,NaN,NaN,NaN
4,4,1.032978e+18,HODLER,/HODL_ER/status/1032977774624079874,2018-08-24 11:08:00+00:00,0.0,0.0,0.0,LATEST PRICES FOR TOP 10 CRYPTOCURRENCIES USD ...,6949.51,5.503697,38235.502385,6947.240108,0.0,0.936,0.064,0.368


In [16]:
#replace empty text with empty string
df['text'] = df['text'].fillna('')

In [17]:
#remove extra columns
df = df.drop(['Unnamed: 0'], axis=1)

In [18]:
#get datetime
df["dt"] = pd.to_datetime(df['timestamp'])
df = df.drop(['timestamp'], axis=1)

In [19]:
#set DateTime 
df['DateTime'] = df['dt'].dt.floor(group_by[0])

In [20]:
#set datetime to index of dataframe
df.index = pd.to_datetime(df.index)

In [21]:
#group by day and take the average of values in that day
df['text'] = df['text'].astype(str)
# df_copy['text'] = df_copy.groupby(['DateTime'])['text'].transform(lambda x : ''.join(x + " "))
tweets_grouped = df.groupby('DateTime').agg(lambda x : ''.join(x + ' '))
#group by day and take the average of values in that day
df_grouped = df.groupby('DateTime').agg(lambda x: x.mean())
df_grouped['text'] = tweets_grouped

In [22]:
tweets_grouped

,text
DateTime,
2018-08-24 00:00:00+00:00,LATEST PRICES FOR TOP 10 CRYPTOCURRENCIES ...
2018-08-25 00:00:00+00:00,Total Market Cap 217213 253599 1 BTC 6738 58 B...
2018-08-26 00:00:00+00:00,Market Cap 216843 015604 BTC Dominance 5368 BT...
2018-08-27 00:00:00+00:00,Total Market Cap 215621 588022 1 BTC 6709 33 B...
2018-08-28 00:00:00+00:00,Total Market Cap 225058 112278 1 BTC 6919 17 B...
...,...
2020-09-03 00:00:00+00:00,...
2020-09-04 00:00:00+00:00,...
2020-09-05 00:00:00+00:00,...


In [23]:
df.head()

,id,fullname,url,replies,likes,retweets,text,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,neg_pol,neu_pol,pos_pol,compound,dt,DateTime
1970-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,NaN,,6936.26,1.001363,6944.466619,6935.011702,NaN,NaN,NaN,NaN,2018-08-24 11:04:00+00:00,2018-08-24 00:00:00+00:00
1970-01-01 00:00:00.000000001,NaN,NaN,NaN,NaN,NaN,NaN,,6944.90,10.379585,72022.107341,6938.823172,NaN,NaN,NaN,NaN,2018-08-24 11:05:00+00:00,2018-08-24 00:00:00+00:00
1970-01-01 00:00:00.000000002,NaN,NaN,NaN,NaN,NaN,NaN,,6949.89,1.264825,8789.841563,6949.454444,NaN,NaN,NaN,NaN,2018-08-24 11:06:00+00:00,2018-08-24 00:00:00+00:00
1970-01-01 00:00:00.000000003,NaN,NaN,NaN,NaN,NaN,NaN,,6950.61,1.948446,13550.262119,6954.393879,NaN,NaN,NaN,NaN,2018-08-24 11:07:00+00:00,2018-08-24 00:00:00+00:00
1970-01-01 00:00:00.000000004,1.032978e+18,HODLER,/HODL_ER/status/1032977774624079874,0.0,0.0,0.0,LATEST PRICES FOR TOP 10 CRYPTOCURRENCIES USD ...,6949.51,5.503697,38235.502385,6947.240108,0.0,0.936,0.064,0.368,2018-08-24 11:08:00+00:00,2018-08-24 00:00:00+00:00


In [24]:
del df
del tweets_grouped

In [25]:
df_grouped.iloc[1000:]

,id,replies,likes,retweets,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,neg_pol,neu_pol,pos_pol,compound,dt,text
DateTime,,,,,,,,,,,,,,


In [26]:
#add tweet volume to df
df_grouped['Tweet_vol'] = grouped_volumes['tweet_vol']
df_grouped['Close'] = close_prices

In [27]:
#remove rows where close is null
df_grouped.dropna(subset = ["Close"], inplace=True)

In [28]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                           ___:  2.5 GiB
                           _22:  2.5 GiB
                    df_grouped:  2.5 GiB
               grouped_volumes: 23.3 KiB
                  close_prices: 23.3 KiB
                           _10:  3.0 KiB
                           _15:  1.9 KiB
                            __:  1.4 KiB
                           _23:  1.4 KiB
                           _i4:  659.0 B


In [29]:
df_grouped.head()

,id,replies,likes,retweets,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,neg_pol,neu_pol,pos_pol,compound,dt,text,Tweet_vol
DateTime,,,,,,,,,,,,,,,
2018-08-24 00:00:00+00:00,1.033070e+18,1.300971,21.330097,7.854369,7027.99,5.263317,36749.800785,6979.691936,0.031903,0.852311,0.115796,0.267373,2018-08-24 17:29:23.615384576+00:00,LATEST PRICES FOR TOP 10 CRYPTOCURRENCIES ...,506.0
2018-08-25 00:00:00+00:00,1.033351e+18,0.125874,4.755245,2.825175,7187.23,3.664420,26193.723043,7096.869104,0.039671,0.853007,0.107336,0.201412,2018-08-25 12:34:15.953488384+00:00,Total Market Cap 217213 253599 1 BTC 6738 58 B...,857.0
2018-08-26 00:00:00+00:00,1.033700e+18,0.842857,19.300000,4.228571,7275.00,2.817791,20387.920017,7233.424594,0.039157,0.820357,0.140486,0.232191,2018-08-26 12:07:15.114155264+00:00,Market Cap 216843 015604 BTC Dominance 5368 BT...,796.0
2018-08-27 00:00:00+00:00,1.034094e+18,6.764331,48.738854,18.745223,7301.01,2.142963,15556.419160,7260.409426,0.037892,0.841885,0.120229,0.228047,2018-08-27 12:22:26.258503168+00:00,Total Market Cap 215621 588022 1 BTC 6709 33 B...,885.0
2018-08-28 00:00:00+00:00,1.034441e+18,2.536842,24.189474,8.205263,7350.00,3.419014,25051.550529,7317.004481,0.037947,0.839863,0.122184,0.226857,2018-08-28 12:10:40.315865088+00:00,Total Market Cap 225058 112278 1 BTC 6919 17 B...,1012.0


In [30]:
#remove unwanted cols
unwanted_cols = ['id', 'retweets', 'likes', 'replies', 'Volume_(Currency)']

if(with_polarity):
    unwanted_cols.append('text')
    
df_grouped = df_grouped.drop(unwanted_cols, axis=1)

In [31]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 457 entries, 2018-08-24 00:00:00+00:00 to 2019-11-23 00:00:00+00:00
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   Close           457 non-null    float64            
 1   Volume_(BTC)    457 non-null    float64            
 2   Weighted_Price  457 non-null    float64            
 3   neg_pol         457 non-null    float64            
 4   neu_pol         457 non-null    float64            
 5   pos_pol         457 non-null    float64            
 6   compound        457 non-null    float64            
 7   dt              457 non-null    datetime64[ns, UTC]
 8   Tweet_vol       457 non-null    float64            
dtypes: datetime64[ns, UTC](1), float64(8)
memory usage: 35.7 KB


In [32]:
folder = "./../datasets/tweets_prices_volumes_sentiment/"+analyser+"/"+group_by+"_datasets" if (with_polarity) else "./../datasets/tweets_prices_volumes_sentiment/cleaned/"+group_by+"_datasets"

if(cleaned and with_polarity):
    folder = folder+"/cleaned"

if not os.path.exists(folder):
    os.makedirs(folder)

In [33]:
folder

'./../datasets/tweets_prices_volumes_sentiment/vader/day_datasets/cleaned'

In [34]:
filename = folder+"/final_data_lag_"+granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"
df_grouped.to_csv(filename)